### Paso 0: importamos librerías

In [2]:
import streamlit as st
import os
from datetime import datetime
import pandas as pd
import joblib
import re
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
import io
import matplotlib.pyplot as plt

## Paso 1: configuración inicial

### 1.1 Cargamos la funciones

In [3]:
# Cargamos el DataFrame Final_XGBosst_data_processed para obtener los datos de los productos
# Pasamos el mes a número ya que el modelo precargado necesita un número entero para procesarlo.
@st.cache_data
def load_data():
    csv_path = "/Users/jesus/Desktop/streamlit - proyecto final/Machine_Learning_Based_Demand_Forecasting/Walmart/data/raw/Final_XGBoost_data_processed.csv"
    try:
        data = pd.read_csv(csv_path)
        # Convertir la columna 'month' de timestamp a número de mes
        if 'month' in data.columns:
            data['month'] = pd.to_datetime(data['month']).dt.month
        return data
    except FileNotFoundError:
        st.error("No se encontró el archivo CSV. Asegúrate de que esté en la ruta correcta.")
        return None

2025-05-16 19:39:25.266 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [4]:
# Precargamos los modelos en caché para predecir de forma más rápida los productos una vez seleccionados.
# Utilizamos joblib para precargar los modelos ya que permite precargar los modelos y utilizarlos de forma más óptima.
@st.cache_resource
def load_model(product_id, store_id):
    model_path = f"/Users/jesus/Desktop/streamlit - proyecto final/Machine_Learning_Based_Demand_Forecasting/Walmart/models/Final_model_XGBOOST_{product_id}_{store_id}.sav"
    if not os.path.exists(model_path):
        st.error(f"No se encontró el modelo en: {model_path}")
        return None
    try:
        model = joblib.load(model_path)
        return model
    except Exception as e:
        st.error(f"Error al cargar el modelo: {str(e)}")
        return None

In [5]:
# Esta función escanea la carpeta models para identificar todos los modelos disponibles
# y extrae los identificadores de productos (product_id) y tiendas (store_id) de sus nombres.

@st.cache_data
def get_available_models():
    model_dir = "/Users/jesus/Desktop/streamlit - proyecto final/Machine_Learning_Based_Demand_Forecasting/Walmart/models"

# Creamos un diccionario porque la idea es obtener tienda por producto: product_to_stores['FOODS_2_197'] = {'CA_1', 'CA_2'}

    product_to_stores = {}

# Inicializa un conjunto vacío asignado a la variable products

    products = set()

#Define una expresión regular para analizar los nombres de los archivos de modelo,
# que siguen el formato Final_model_XGBOOST_<product_id>_<store_id>.sav.

    model_pattern = re.compile(r"Final_model_XGBOOST_(.+)_([A-Z]{2}_\d+)\.sav")
    
# Extrae product_id y store_id de los nombres de archivo (por ejemplo,
# de Final_model_XGBOOST_FOODS_2_197_CA_1.sav obtiene FOODS_2_197 y CA_1).

    try:
        for filename in os.listdir(model_dir):
            match = model_pattern.match(filename)
            if match:
                product_id = match.group(1)
                store_id = match.group(2)
                products.add(product_id)
                if product_id not in product_to_stores:
                    product_to_stores[product_id] = set()
                product_to_stores[product_id].add(store_id)
        return sorted(products), product_to_stores
    except FileNotFoundError:
        st.error(f"No se encontró la carpeta de modelos en: {model_dir}")
        return [], {}

2025-05-16 19:39:25.280 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


## Paso 2: autentificación con nombre


### 2.1 cargamos el logo de Walmart

In [6]:
# Mostrar logo de Walmart al inicio
walmart_logo_path = "/Users/jesus/Desktop/streamlit - proyecto final/Machine_Learning_Based_Demand_Forecasting/Walmart/data/raw/Walmart_logo.svg.png"
if os.path.exists(walmart_logo_path):
    st.image(walmart_logo_path, use_container_width=True)

2025-05-16 19:39:25.285 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 19:39:25.319 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 19:39:25.401 
  command:

    streamlit run /Users/jesus/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-05-16 19:39:25.401 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


### 2.2 cargamos el autentificador de usuario

In [ ]:
# Inicializar el estado de la sesión para la autenticación
if "username" not in st.session_state:
    st.session_state["username"] = None
if "login_time" not in st.session_state:
    st.session_state["login_time"] = None
if "selected_state_full" not in st.session_state:
    st.session_state["selected_state_full"] = None

# Si el usuario no ha puesto un nombre, mostrar el formulario
if st.session_state["username"] is None:
    st.markdown("<h3 style='text-align: center;'>Bienvenido!</h3>", unsafe_allow_html=True)
    st.markdown("<p style='text-align: center;'>Por favor, introduce tu nombre para continuar:</p>", unsafe_allow_html=True)
    
    username = st.text_input("Nombre de usuario", key="username_input")
    
    if st.button("Ingresar"):
        if username.strip() == "":
            st.error("Por favor, ingresa un nombre válido.")
        else:
            # Registrar el nombre y la hora de ingreso
            st.session_state["username"] = username
            st.session_state["login_time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            st.rerun()

# Si el usuario ya ingresó un nombre, mostrar el resto de la app
if st.session_state["username"] is not None:

# Mostrar mensaje de bienvenida con nombre, fecha y hora
    st.markdown(
        f"""
        <p style='text-align: center; color: #0071CE;'>
            Bienvenido, {st.session_state["username"]}. Has ingresado el {st.session_state["login_time"]}.
        </p>
        """,
        unsafe_allow_html=True
    )

### 2.3 creamos el desplegable

In [ ]:

# Creamos el desplegable para seleccionar que apartado queremos ver
mode = st.selectbox(
        "Selecciona el modo:",
        ["Predicción de la demanda", "Informe de la predicción"],
        key="mode_select"
    )

### 2.4 creamos el título de "Predicción de la demanda" para que nos aparezca de forma más visible en que apartado estamos trabajando

In [ ]:
st.markdown(
            """
            <h1 style='color: #0071CE; text-align: center; white-space: nowrap; margin-top: 0; margin-bottom: 20px;'>
                Predicción de la demanda
            </h1>
            """,
            unsafe_allow_html=True
        )

## Paso 3: predicción de la demanda

### 3.1 obtenemos los productos, las tiendas y los estados

In [ ]:
if mode == "Predicción de la demanda":

# Obtener productos y mapeo de productos a tiendas desde los nombres de los modelos
# Cargamos la función para obtener del nombre de los modelos el identificador del producto y la tienda

        products, product_to_stores = get_available_models()
        if not products or not product_to_stores:
            st.error("No se encontraron modelos válidos. Verifica la carpeta de modelos.")
            st.stop()
        
         # Mapa de estados
        state_mapping = {
            'CA': 'California',
            'TX': 'Texas',
            'WI': 'Wisconsin'
        }

# Creamos el sidebar con los titulos que vamos a visualizar y los datos que van a aparecer dentro de cada desplegable

        st.sidebar.header("Parámetros a predecir")

# Botón para cerrar sesión
        if st.sidebar.button("Cerrar sesión"):
            st.session_state["username"] = None
            st.session_state["login_time"] = None
            st.session_state["selected_state_full"] = None
            st.rerun()

# Selección de producto

        selected_product = st.sidebar.selectbox("Selecciona un producto:", products, key="product_select")

# Obtener tiendas disponibles para el producto seleccionado

        available_stores = sorted(product_to_stores.get(selected_product, []))
        available_stores_display = [f"Tienda {store.split('_')[1]}" for store in available_stores]
        
# Selección de tienda (formato "Tienda X"). Como hemos desglosado el nombre del producto para obtener la tienda,
# para que no aparezca solo "2" o "1" añadimos "Tienda" en primer lugar para que sea más claro y comprensible

        selected_store_display = st.sidebar.selectbox("Selecciona una tienda:", available_stores_display, key="store_select") if available_stores_display else None
        selected_store = available_stores[available_stores_display.index(selected_store_display)] if selected_store_display else None

# Mostrar el estado de la tienda seleccionada

        if selected_store:
            store_state_code = selected_store.split('_')[0]
            store_state_full = state_mapping.get(store_state_code, "Desconocido")
            st.sidebar.write(f"Estado: {store_state_full}")
        else:
            st.sidebar.warning("No hay tiendas disponibles para el producto seleccionado.")
            st.sidebar.write("Estado: No seleccionado")

# Selección de mes y año (fijado a junio de 2016). Como solo hemos predecido eso,
# fijamos el mes a predecir que es con el que podemos comparar

        st.sidebar.subheader("Período de predicción")
        selected_month = st.sidebar.selectbox("Mes", ["Junio"], disabled=True, key="month_select")
        selected_year = st.sidebar.selectbox("Año", [2016], disabled=True, key="year_select")

### 3.2 Creamos un mapa interactivo de EEUU para que reaccione con la tienda seleccionada

In [ ]:
if selected_store:
            # Determinar el estado seleccionado a partir de la tienda
            store_state_code = selected_store.split('_')[0]
            selected_state_full = state_mapping.get(store_state_code, "Desconocido")

            # Crear mapa interactivo con Plotly (se cargan todos los estados aunque no es necesario)
            us_states = [
                {'name': 'Alabama', 'code': 'AL'}, {'name': 'Alaska', 'code': 'AK'}, {'name': 'Arizona', 'code': 'AZ'},
                {'name': 'Arkansas', 'code': 'AR'}, {'name': 'California', 'code': 'CA'}, {'name': 'Colorado', 'code': 'CO'},
                {'name': 'Connecticut', 'code': 'CT'}, {'name': 'Delaware', 'code': 'DE'}, {'name': 'Florida', 'code': 'FL'},
                {'name': 'Georgia', 'code': 'GA'}, {'name': 'Hawaii', 'code': 'HI'}, {'name': 'Idaho', 'code': 'ID'},
                {'name': 'Illinois', 'code': 'IL'}, {'name': 'Indiana', 'code': 'IN'}, {'name': 'Iowa', 'code': 'IA'},
                {'name': 'Kansas', 'code': 'KS'}, {'name': 'Kentucky', 'code': 'KY'}, {'name': 'Louisiana', 'code': 'LA'},
                {'name': 'Maine', 'code': 'ME'}, {'name': 'Maryland', 'code': 'MD'}, {'name': 'Massachusetts', 'code': 'MA'},
                {'name': 'Michigan', 'code': 'MI'}, {'name': 'Minnesota', 'code': 'MN'}, {'name': 'Mississippi', 'code': 'MS'},
                {'name': 'Missouri', 'code': 'MO'}, {'name': 'Montana', 'code': 'MT'}, {'name': 'Nebraska', 'code': 'NE'},
                {'name': 'Nevada', 'code': 'NV'}, {'name': 'New Hampshire', 'code': 'NH'}, {'name': 'New Jersey', 'code': 'NJ'},
                {'name': 'New Mexico', 'code': 'NM'}, {'name': 'New York', 'code': 'NY'}, {'name': 'North Carolina', 'code': 'NC'},
                {'name': 'North Dakota', 'code': 'ND'}, {'name': 'Ohio', 'code': 'OH'}, {'name': 'Oklahoma', 'code': 'OK'},
                {'name': 'Oregon', 'code': 'OR'}, {'name': 'Pennsylvania', 'code': 'PA'}, {'name': 'Rhode Island', 'code': 'RI'},
                {'name': 'South Carolina', 'code': 'SC'}, {'name': 'South Dakota', 'code': 'SD'}, {'name': 'Tennessee', 'code': 'TN'},
                {'name': 'Texas', 'code': 'TX'}, {'name': 'Utah', 'code': 'UT'}, {'name': 'Vermont', 'code': 'VT'},
                {'name': 'Virginia', 'code': 'VA'}, {'name': 'Washington', 'code': 'WA'}, {'name': 'West Virginia', 'code': 'WV'},
                {'name': 'Wisconsin', 'code': 'WI'}, {'name': 'Wyoming', 'code': 'WY'}
            ]
            state_df = pd.DataFrame(us_states)
            state_df['color'] = state_df['name'].apply(lambda x: 'Seleccionado' if x == selected_state_full else 'No seleccionado')

            # Crear el mapa de EEUU con px.choropleth. Haremos que se ilumine con la tienda seleccionada vinculada a cada estado
            fig = px.choropleth(
                state_df,
                locations='code',
                locationmode="USA-states",
                color='color',
                scope="usa",
                color_discrete_map={
                    'Seleccionado': '#0071CE',
                    'No seleccionado': '#E0E0E0'
                },
                labels={'color': 'Estado'},
                title=f"Estado Seleccionado: {selected_state_full}"
            )
            fig.update_layout(
                plot_bgcolor='rgba(240, 240, 240, 1)',
                paper_bgcolor='rgba(240, 240, 240, 1)',
                font=dict(color='black', family="Arial, sans-serif"),
                title=dict(
                    font=dict(size=20),
                    x=0.5,
                    xanchor='center'
                ),
                geo=dict(
                    bgcolor='rgba(240, 240, 240, 1)',
                    lakecolor='rgba(255, 255, 255, 1)',
                    landcolor='rgba(200, 200, 200, 0.5)',
                    subunitcolor='rgba(0, 0, 0, 0.8)',
                    showlakes=True,
                    showsubunits=True,
                    showland=True
                ),
                showlegend=False,
                margin=dict(l=10, r=10, t=50, b=10)
            )
            fig.update_traces(showscale=False)

            
            st.plotly_chart(fig, use_container_width=True, key="map")

## Paso 4: cargamos los modelos y predecimos la demanda (con gráficos)

In [ ]:
#Cargamos el modelo correspondiente al producto y la tienda seleccionado

if selected_store:
    # Cargar el modelo correspondiente
    model = load_model(selected_product, selected_store)
    if model is not None:
        # Cargar datos del CSV
        data = load_data()
        if data is None:
            st.stop()

            # Preparar datos para la predicción (junio de 2016)
            try:
                    
                # Filtrar datos para la tienda y producto seleccionados
                filtered_data = data[(data['store_id'] == selected_store) & (data['item_id'] == selected_product)].copy()
                    
                if filtered_data.empty:
                    st.warning("No se encontraron datos para la combinación de tienda y producto seleccionada en el CSV.")
                else:
                        
                    # Definir las columnas de características esperadas por el modelo
                    feature_columns = [
                            'event_name_1', 'snap', 'sell_price', 'lag_1', 'lag_2', 'lag_3', 
                            'lag_6', 'lag_12', 'rolling_mean_3', 'year', 
                            'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 
                            'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12'
                    ]
                        
                    # Verificar que todas las columnas requeridas estén presentes
                    missing_columns = [col for col in feature_columns if col not in filtered_data.columns]
                    if missing_columns:
                        st.error(f"Faltan las siguientes columnas en los datos: {missing_columns}")
                        st.stop()
                        
                    # Preparar datos para la predicción de junio 2016
                    last_row = filtered_data.iloc[-1]
                    prediction_data = pd.DataFrame({
                            'event_name_1': [last_row['event_name_1']],
                            'snap': [last_row['snap']],
                            'sell_price': [last_row['sell_price']],
                            'lag_1': [last_row['sales']],
                            'lag_2': [filtered_data['sales'].iloc[-2] if len(filtered_data) >= 2 else filtered_data['sales'].mean()],
                            'lag_3': [filtered_data['sales'].iloc[-3] if len(filtered_data) >= 3 else filtered_data['sales'].mean()],
                            'lag_6': [filtered_data['sales'].iloc[-6] if len(filtered_data) >= 6 else filtered_data['sales'].mean()],
                            'lag_12': [filtered_data['sales'].iloc[-12] if len(filtered_data) >= 12 else filtered_data['sales'].mean()],
                            'rolling_mean_3': [filtered_data['sales'].tail(3).mean()],
                            'year': [2016],
                            'month_1': [0], 'month_2': [0], 'month_3': [0], 'month_4': [0], 'month_5': [0],
                            'month_6': [1], 'month_7': [0], 'month_8': [0], 'month_9': [0], 'month_10': [0],
                            'month_11': [0], 'month_12': [0]
                    })
                        
                    # Realizar la predicción para junio 2016
                    predicted_log = model.predict(prediction_data[feature_columns])[0]
                    predicted_demand = np.expm1(predicted_log)
                        
                    # Obtener demanda real (usamos el último mes disponible, e.g., mayo 2016)
                    real_demand = last_row['sales']
                        
                    # Crear gráfico de barras 2D con efecto pseudo-3D
                    fig = go.Figure()

                    # Definir posiciones y dimensiones para las barras
                    bar_width = 0.3
                    depth_offset = 0.05 * max(real_demand, predicted_demand)

                    # Barra principal para demanda real (cara frontal)
                    fig.add_trace(go.Bar(
                            x=[0],
                            y=[real_demand],
                            name='Demanda Real (Junio 2016)',
                            marker_color='rgb(55, 83, 109)',
                            marker_line_color='rgb(8, 48, 107)',
                            marker_line_width=2,
                            opacity=0.9,
                            width=bar_width,
                            text=[f"{real_demand:.2f}"],
                            textposition='auto'
                    ))

                    # Cara superior para demanda real
                    fig.add_trace(go.Scatter(
                            x=[-bar_width / 2, bar_width / 2, bar_width / 2 + 0.1, -bar_width / 2 + 0.1, -bar_width / 2],
                            y=[real_demand, real_demand, real_demand + depth_offset, real_demand + depth_offset, real_demand],
                            mode='lines',
                            fill='toself',
                            fillcolor='rgb(75, 103, 129)',
                            line=dict(color='rgb(8, 48, 107)', width=2),
                            showlegend=False,
                            hoverinfo='skip'
                    ))

                    # Cara lateral para demanda real
                    fig.add_trace(go.Scatter(
                            x=[bar_width / 2, bar_width / 2 + 0.1, bar_width / 2 + 0.1, bar_width / 2, bar_width / 2],
                            y=[0, 0, real_demand + depth_offset, real_demand, 0],
                            mode='lines',
                            fill='toself',
                            fillcolor='rgb(45, 63, 89)',
                            line=dict(color='rgb(8, 48, 107)', width=2),
                            showlegend=False,
                            hoverinfo='skip'
                    ))

                    # Barra principal para demanda predicha
                    fig.add_trace(go.Bar(
                            x=[1],
                            y=[predicted_demand],
                            name='Demanda Predicha (Junio 2016)',
                            marker_color='rgb(255, 140, 0)',
                            marker_line_color='rgb(204, 102, 0)',
                            marker_line_width=2,
                            opacity=0.9,
                            width=bar_width,
                            text=[f"{predicted_demand:.2f}"],
                            textposition='auto'
                    ))

                    # Cara superior para demanda predicha
                    fig.add_trace(go.Scatter(
                            x=[1 - bar_width / 2, 1 + bar_width / 2, 1 + bar_width / 2 + 0.1, 1 - bar_width / 2 + 0.1, 1 - bar_width / 2],
                            y=[predicted_demand, predicted_demand, predicted_demand + depth_offset, predicted_demand + depth_offset, predicted_demand],
                            mode='lines',
                            fill='toself',
                            fillcolor='rgb(255, 160, 50)',
                            line=dict(color='rgb(204, 102, 0)', width=2),
                            showlegend=False,
                            hoverinfo='skip'
                    ))

                        # Cara lateral para demanda predicha
                    fig.add_trace(go.Scatter(
                            x=[1 + bar_width / 2, 1 + bar_width / 2 + 0.1, 1 + bar_width / 2 + 0.1, 1 + bar_width / 2, 1 + bar_width / 2],
                            y=[0, 0, predicted_demand + depth_offset, predicted_demand, 0],
                            mode='lines',
                            fill='toself',
                            fillcolor='rgb(215, 110, 0)',
                            line=dict(color='rgb(204, 102, 0)', width=2),
                            showlegend=False,
                            hoverinfo='skip'
                    ))

                    # Actualizar el diseño
                    fig.update_layout(
                            title={
                                'text': "Demanda Real vs. Predicha para Junio 2016",
                                'y': 0.95,
                                'x': 0.5,
                                'xanchor': 'center',
                                'yanchor': 'top',
                                'font': dict(size=20, color='white', family="Arial, sans-serif")
                            },
                            xaxis_title="Mes",
                            yaxis_title="Demanda",
                            barmode='group',
                            bargap=0.2,
                            bargroupgap=0.1,
                            plot_bgcolor='rgba(0,0,0,0)',
                            paper_bgcolor='rgba(30,30,60,1)',
                            font=dict(color='white', family="Arial, sans-serif"),
                            xaxis=dict(
                                title=dict(text="Mes", font=dict(size=16)),
                                tickfont=dict(size=14),
                                tickvals=[0, 1],
                                ticktext=['Demanda Real', 'Demanda Predicha'],
                                gridcolor='rgba(255,255,255,0.1)'
                            ),
                            yaxis=dict(
                                title=dict(text="Demanda", font=dict(size=16)),
                                tickfont=dict(size=14),
                                gridcolor='rgba(255,255,255,0.1)',
                                range=[0, max(real_demand, predicted_demand) * 1.2]
                            ),
                            legend=dict(
                                x=0.75,
                                y=1.1,
                                font=dict(size=12, color='white'),
                                bgcolor='rgba(0,0,0,0)',
                                orientation='h'
                            ),
                            height=600,
                            margin=dict(l=50, r=50, t=100, b=50)
                        )

                    st.plotly_chart(fig, use_container_width=True)
                        
            except Exception as e:
                    st.error(f"Error al realizar la predicción: {str(e)}")
        else:
            st.info("Por favor, selecciona una tienda para realizar la predicción.")


## Paso 5: Obtener informe de predicción de la demanda

In [ ]:
# Da error porque está todo identado de forma que una opción active otra, pero lo separo para que se entiendan los pasos
# El objetivo es ir en este orden: Estados --> Tiendas --> Productos.
# De esta forma, al elegir el estado aparecerán solo las tiendas de ese estado y al seleccionar la tienda solo los productos
# que corresponden a esa tienda. En caso de querer elegir todo a la vez, hay que filtrarlo de forma que los productos NO
# presentes en una tienda en cuestión no aparezcan representados y de error

elif mode == "Informe de la predicción":

    st.markdown(
            """
            <h1 style='color: #0071CE; text-align: center; white-space: nowrap; margin-top: 0; margin-bottom: 20px;'>
                Informe de la predicción
            </h1>
            """,
            unsafe_allow_html=True
        )
    # Obtener productos y mapeo de productos a tiendas
        products, product_to_stores = get_available_models()
        if not products or not product_to_stores:
            st.error("No se encontraron modelos válidos. Verifica la carpeta de modelos.")
            st.stop()

        # Mapa de estados
        state_mapping = {
            'CA': 'California',
            'TX': 'Texas',
            'WI': 'Wisconsin'
        }
        all_states = sorted(state_mapping.values())

        # Selección de estado
        st.subheader("Selecciona estado, tiendas y productos para el informe")
        selected_state_full = st.selectbox(
            "Estado:",
            all_states,
            key="report_state"
        )

        # Filtrar tiendas disponibles para el estado seleccionado
        state_code = next((code for code, name in state_mapping.items() if name == selected_state_full), None)
        available_stores = []
        store_to_id = {}
        if state_code:
            for product in products:
                for store in product_to_stores.get(product, []):
                    if store.startswith(state_code):
                        store_number = store.split('_')[1]
                        store_display = f"Tienda {store_number} ({selected_state_full})"
                        if store_display not in available_stores:
                            available_stores.append(store_display)
                            store_to_id[store_display] = store
        available_stores = sorted(available_stores)

        # Selección de tiendas
        selected_stores_display = st.multiselect(
            "Tiendas:",
            available_stores,
            key="report_stores"
        )
        selected_stores = [store_to_id[store_display] for store_display in selected_stores_display]

        # Filtrar productos disponibles para las tiendas seleccionadas
        available_products = set()
        for store in selected_stores:
            for product in products:
                if store in product_to_stores.get(product, []):
                    available_products.add(product)
        available_products = sorted(available_products)

        # Selección de productos
        selected_products = st.multiselect(
            "Productos:",
            available_products,
            key="report_products"
        )

## Paso 6: generar informe en PDF de los datos preseleccionados

- Los pasos son, una vez seleccionado el/los estado/s, tienda/s y producto/s se carguen en un informe en pdf.
- Para ello, debemos cargar una tabla con los datos tanto reales como predichos y la diferencia en % de error.
- Además, se han seleccionado colores y formatos para que sean más visibles y profesionales.
- Como también queremos que se carguen los gráficos, hay que codificarlo también en base a los datos seleccionados.

In [ ]:
if st.button("Generar Informe"):
            if not selected_state_full or not selected_stores or not selected_products:
                st.error("Por favor, selecciona un estado, al menos una tienda y al menos un producto.")
            else:
                import tempfile
                import time
                import matplotlib.pyplot as plt

                # Crear directorio temporal para imágenes
                with tempfile.TemporaryDirectory() as tmpdirname:
                    # Crear buffer para el PDF
                    buffer = io.BytesIO()
                    doc = SimpleDocTemplate(buffer, pagesize=letter, rightMargin=50, leftMargin=50, topMargin=50, bottomMargin=50)
                    story = []
                    styles = getSampleStyleSheet()

                    # Estilos personalizados
                    title_style = ParagraphStyle(
                        'Title',
                        parent=styles['Title'],
                        fontSize=16,
                        textColor=colors.HexColor('#0071CE'),
                        spaceAfter=20,
                        alignment=1  # Centrado
                    )
                    heading_style = ParagraphStyle(
                        'Heading2',
                        parent=styles['Heading2'],
                        fontSize=12,
                        spaceAfter=10
                    )
                    normal_style = styles['Normal']

                    # Título del informe
                    story.append(Paragraph(f"Informe de predicciones - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", title_style))
                    story.append(Paragraph(f"Usuario: {st.session_state['username']}", normal_style))
                    story.append(Spacer(1, 12))

                    # Tabla de predicciones
                    table_data = [['Producto', 'Estado', 'Tienda', 'Demanda real (mayo 2016)', 'Demanda predicha (junio 2016)', '% Error']]
                    image_paths = []

                    for product in selected_products:
                        for store in selected_stores:
                            # Verificar si el producto se vende en la tienda
                            if store not in product_to_stores.get(product, []):
                                continue  # Ignorar combinación inválida

                            # Verificar que la tienda pertenece al estado seleccionado
                            store_state_code = store.split('_')[0]
                            if store_state_code != state_code:
                                continue  # Ignorar si la tienda no coincide con el estado

                            # Cargar modelo
                            model = load_model(product, store)
                            if model is None:
                                continue

                            # Cargar datos
                            data = load_data()
                            if data is None:
                                continue

                            # Preparar datos para la predicción
                            filtered_data = data[(data['store_id'] == store) & (data['item_id'] == product)].copy()
                            if filtered_data.empty:
                                continue

                            # Definir columnas de características
                            feature_columns = [
                                'event_name_1', 'snap', 'sell_price', 'lag_1', 'lag_2', 'lag_3',
                                'lag_6', 'lag_12', 'rolling_mean_3', 'year',
                                'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
                                'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12'
                            ]
                            missing_columns = [col for col in feature_columns if col not in filtered_data.columns]
                            if missing_columns:
                                continue

                            # Preparar datos para predicción
                            last_row = filtered_data.iloc[-1]
                            prediction_data = pd.DataFrame({
                                'event_name_1': [last_row['event_name_1']],
                                'snap': [last_row['snap']],
                                'sell_price': [last_row['sell_price']],
                                'lag_1': [last_row['sales']],
                                'lag_2': [filtered_data['sales'].iloc[-2] if len(filtered_data) >= 2 else filtered_data['sales'].mean()],
                                'lag_3': [filtered_data['sales'].iloc[-3] if len(filtered_data) >= 3 else filtered_data['sales'].mean()],
                                'lag_6': [filtered_data['sales'].iloc[-6] if len(filtered_data) >= 6 else filtered_data['sales'].mean()],
                                'lag_12': [filtered_data['sales'].iloc[-12] if len(filtered_data) >= 12 else filtered_data['sales'].mean()],
                                'rolling_mean_3': [filtered_data['sales'].tail(3).mean()],
                                'year': [2016],
                                'month_1': [0], 'month_2': [0], 'month_3': [0], 'month_4': [0], 'month_5': [0],
                                'month_6': [1], 'month_7': [0], 'month_8': [0], 'month_9': [0], 'month_10': [0],
                                'month_11': [0], 'month_12': [0]
                            })

                            # Realizar predicción
                            try:
                                predicted_log = model.predict(prediction_data[feature_columns])[0]
                                predicted_demand = np.expm1(predicted_log)
                                real_demand = last_row['sales']
                            except Exception:
                                continue

                            # Calcular % de error
                            error_percent = abs(real_demand - predicted_demand) / real_demand * 100 if real_demand != 0 else 0

                            # Añadir a la tabla
                            table_data.append([
                                product,
                                selected_state_full,
                                f"Tienda {store.split('_')[1]}",
                                f"{real_demand:.2f}",
                                f"{predicted_demand:.2f}",
                                f"{error_percent:.2f}%"
                            ])

                            # Generar gráfico con matplotlib
                            fig, ax = plt.subplots(figsize=(8, 6), facecolor='#1E1E3C')
                            ax.set_facecolor('#1E1E3C')
                            bar_width = 0.35
                            x = [0, 1]
                            bars = ax.bar([i - bar_width/2 for i in x], [real_demand, predicted_demand], 
                                         bar_width, color=['#37536D', '#FF8C00'], edgecolor='#08306B')
                            
                            # Añadir texto encima de las barras
                            for bar in bars:
                                height = bar.get_height()
                                ax.text(bar.get_x() + bar.get_width()/2., height,
                                       f'{height:.2f}', ha='center', va='bottom', color='white')

                            # Configurar ejes
                            ax.set_xticks(x)
                            ax.set_xticklabels(['Demanda real', 'Demanda predicha'], color='white')
                            ax.set_ylabel('Demanda', color='white')
                            ax.set_title(f'Demanda Real vs. Predicha - {product}, {selected_state_full}, Tienda {store.split("_")[1]}', 
                                        color='white', pad=20)
                            ax.tick_params(axis='y', colors='white')
                            ax.spines['bottom'].set_color('white')
                            ax.spines['left'].set_color('white')
                            ax.spines['top'].set_visible(False)
                            ax.spines['right'].set_visible(False)

                            # Guardar gráfico como imagen
                            img_path = os.path.join(tmpdirname, f"plot_{product}_{store}_{len(image_paths)}.png")
                            try:
                                plt.savefig(img_path, format='png', bbox_inches='tight', facecolor='#1E1E3C', edgecolor='none')
                                time.sleep(1)
                                if os.path.exists(img_path):
                                    image_paths.append(img_path)
                                else:
                                    st.warning(f"No se generó la imagen para {product}, {selected_state_full}, Tienda {store.split('_')[1]}")
                            except Exception as e:
                                st.warning(f"Error al generar imagen para {product}, {selected_state_full}, Tienda {store.split('_')[1]}: {str(e)}")
                            finally:
                                plt.close(fig)

                    # Crear tabla en PDF
                    if len(table_data) > 1:  # Si hay datos
                        table = Table(table_data)
                        table.setStyle(TableStyle([
                            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#0071CE')),
                            ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
                            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                            ('FONTSIZE', (0, 0), (-1, 0), 10),
                            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                            ('BACKGROUND', (0, 1), (-1, -1), colors.white),
                            ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
                            ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
                            ('FONTSIZE', (0, 1), (-1, -1), 8),
                            ('GRID', (0, 0), (-1, -1), 1, colors.black),
                            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE')
                        ]))
                        story.append(Paragraph("Resumen de predicciones", heading_style))
                        story.append(table)
                        story.append(Spacer(1, 12))

                        # Añadir gráficos
                        if image_paths:
                            story.append(Paragraph("Gráficos de Demanda", heading_style))
                            for img_path in image_paths:
                                if os.path.exists(img_path):
                                    img = Image(img_path, width=450, height=300)
                                    img.hAlign = 'CENTER'
                                    story.append(img)
                                    story.append(Spacer(1, 12))

                        # Generar PDF
                        doc.build(story)
                        buffer.seek(0)

                        # Descargar PDF directamente
                        st.download_button(
                            label="Descargar informe",
                            data=buffer,
                            file_name=f"Informe_predicciones_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf",
                            mime="application/pdf",
                            key="download_pdf"
                        )
                    else:
                        st.warning("No se encontraron datos válidos para las combinaciones seleccionadas.")

## Explicación detallada paso a paso de la aplicación streamlit de Walmart

### 1. Importación de bibliotecas

- El primer paso del desarrollo de esta aplicación consiste en importar todas las bibliotecas de Python necesarias para que funcione correctamente. Este paso es crucial porque establece las herramientas que se utilizarán para procesar datos, crear visualizaciones, manejar modelos de machine learning y generar informes. A continuación, desgloso cada biblioteca y su propósito específico:



- Streamlit (import streamlit as st): Esta es la biblioteca principal que permite construir una interfaz web interactiva. Con Streamlit, los usuarios pueden interactuar con la aplicación a través de widgets como desplegables, botones y campos de texto, y ver resultados en tiempo real sin necesidad de escribir código backend complejo. Por ejemplo, se usa st.selectbox para seleccionar opciones y st.image para mostrar el logotipo de Walmart.



- Pandas (import pandas as pd): Esta biblioteca es esencial para la manipulación de datos tabulares. En la aplicación, se utiliza para cargar un archivo CSV (Final_XGBoost_data_processed.csv) que contiene datos históricos de ventas, filtrarlos según las selecciones del usuario (como producto y tienda) y preparar las características necesarias para las predicciones.



- NumPy (import numpy as np): Complementa a Pandas al proporcionar funciones matemáticas y operaciones con arreglos numéricos. Por ejemplo, se usa np.expm1 para transformar predicciones logarítmicas de vuelta a su escala original, ya que el modelo XGBoost trabaja con datos transformados logarítmicamente.



- Plotly (import plotly.graph_objects as go y import plotly.express as px): Esta biblioteca permite crear gráficos interactivos que los usuarios pueden explorar directamente en la interfaz de Streamlit. Con plotly.express.choropleth, se genera un mapa de Estados Unidos que destaca el estado seleccionado, mientras que plotly.graph_objects se usa para crear gráficos de barras con efectos pseudo-3D que comparan la demanda real y predicha.



- Matplotlib (import matplotlib.pyplot as plt): Aunque Plotly maneja visualizaciones interactivas, Matplotlib se utiliza para generar gráficos estáticos que se incrustan en el informe PDF. Por ejemplo, se crean gráficos de barras para cada combinación de producto y tienda, que luego se guardan como imágenes PNG con plt.savefig.



- Reportlab (from reportlab.lib.pagesizes import letter, etc.): Esta biblioteca es clave para generar documentos PDF personalizados. Se usa para estructurar el informe de predicción, incluyendo un encabezado con el título y la marca de tiempo, una tabla con resultados y las imágenes de los gráficos generados por Matplotlib.



- Joblib (import joblib): Facilita la carga de modelos de machine learning preentrenados (archivos .sav) de manera eficiente. Esto es importante porque los modelos XGBoost son grandes y cargarlos repetidamente sin Joblib sería lento e ineficiente.



#### Bibliotecas estándar de Python:

- os: Permite interactuar con el sistema operativo, como listar archivos en un directorio para encontrar los modelos disponibles.
datetime: Se usa para registrar la marca de tiempo del inicio de sesión del usuario y nombrar archivos PDF con fechas únicas.
re: Proporciona expresiones regulares para extraer información (como identificadores de productos y tiendas) de los nombres de los archivos de modelos.

- io: Maneja buffers de memoria, como BytesIO, para generar y descargar el PDF directamente desde la memoria sin necesidad de guardarlo en disco.



- Cada una de estas bibliotecas se importa al inicio del script Python, generalmente en las primeras líneas, para que estén disponibles en todo el programa.

## 2. Configuración Inicial

- Antes de que la aplicación interactúe con el usuario, se deben preparar los datos y los modelos. Este paso define funciones clave que optimizan el rendimiento y aseguran que todo esté listo para las predicciones y visualizaciones. Aquí está el desglose detallado:



Carga de datos (def load_data()):

- Propósito: Esta función lee el archivo CSV Final_XGBoost_data_processed.csv, que contiene datos históricos de ventas procesados para el modelo XGBoost.

- Detalles del archivo: El CSV incluye columnas como identificadores de productos (por ejemplo, FOODS_2_197), tiendas (por ejemplo, CA_1), fechas (columna month), y características como retrasos (lags), medias móviles y variables dummy.

- Procesamiento: La columna month originalmente podría estar en formato de marca de tiempo (timestamp), pero se convierte a un entero (1 para enero, 2 para febrero, etc.) usando una transformación como pd.to_datetime().dt.month. Esto asegura compatibilidad con el modelo de predicción.

- Optimización: La función está decorada con @st.cache_data, una característica de Streamlit que guarda los datos en caché. Esto significa que el CSV solo se carga una vez al inicio y no se recarga innecesariamente en cada interacción del usuario, mejorando el rendimiento.



Carga de modelos (def load_model(model_path)):

- Propósito: Carga un modelo XGBoost preentrenado desde un archivo .sav especificado por la ruta model_path.
Estructura de los modelos: Cada modelo está nombrado según la combinación de producto y tienda, por ejemplo, Final_model_XGBOOST_FOODS_2_197_CA_1.sav. Esto indica que el modelo predice la demanda para el producto FOODS_2_197 en la tienda CA_1.

- Implementación: Usa joblib.load(model_path) para leer el archivo y devolver el modelo listo para hacer predicciones.
Optimización: Decorada con @st.cache_resource, esta función asegura que los modelos se carguen solo cuando cambie la ruta del archivo, reduciendo el uso de memoria y tiempo.



Modelos disponibles (def get_available_models(model_dir)):

- Propósito: Escanea un directorio (model_dir) para identificar todos los modelos disponibles y organizar la información en un diccionario útil.

Proceso:
- Usa os.listdir(model_dir) para listar todos los archivos en el directorio.
- Filtra los archivos que terminan en .sav (modelos válidos).
- Usa una expresión regular (re.search) para extraer el producto (por ejemplo, FOODS_2_197) y la tienda (por ejemplo, CA_1) de cada nombre de archivo. Un ejemplo de expresión regular sería: r"Final_model_XGBOOST_(.+?)_(.+?)\.sav".
- Crea un diccionario product_to_stores donde las claves son productos y los valores son conjuntos de tiendas disponibles. Por ejemplo: product_to_stores['FOODS_2_197'] = {'CA_1', 'CA_2'}.


- Uso: Este diccionario se utiliza más adelante para poblar los desplegables de selección de productos y tiendas en la interfaz, asegurando que solo se muestren combinaciones válidas.

- Este paso es como preparar los ingredientes antes de cocinar: asegura que todo esté listo y optimizado para los pasos siguientes.

### 3. Autenticación de Usuario

- La autenticación agrega una capa de personalización y seguridad básica. Aquí está cómo se implementa con detalle:



Visualización del logotipo:

- Usa st.image("walmart_logo.png", width=200) para mostrar el logotipo de Walmart al inicio de la aplicación. El archivo walmart_logo.png debe estar en el mismo directorio que el script o en una ruta especificada.


Entrada de nombre de usuario:

- Estado inicial: Se verifica si existe un nombre de usuario en st.session_state.user_name. Si no existe (es None), significa que el usuario no ha iniciado sesión.

- Campo de texto: Se muestra un campo con st.text_input("Por favor, ingresa tu nombre") donde el usuario escribe su nombre.
Validación: Si el usuario deja el campo vacío o escribe solo espacios, se muestra un mensaje de error con st.error("El nombre no puede estar vacío"), y no se avanza hasta que se ingrese un nombre válido.


Gestión de sesión:

- Cuando se ingresa un nombre válido (por ejemplo, "Juan"), se almacena en st.session_state.user_name = "Juan".
Se registra la marca de tiempo del inicio de sesión con st.session_state.login_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S").

- La aplicación se vuelve a ejecutar automáticamente (usando st.rerun()), y ahora muestra un mensaje como: st.write(f"Bienvenido, Juan. Has ingresado el 2023-10-15 14:30:00").


Cierre de sesión:

- En la barra lateral (st.sidebar), se agrega un botón con st.sidebar.button("Cerrar sesión").

- Si se presiona, se eliminan las claves de st.session_state con del st.session_state.user_name y del st.session_state.login_time, y la aplicación se reinicia con st.rerun() para volver al estado de autenticación inicial.

- Este sistema asegura que la aplicación sea personalizada y que solo usuarios autenticados accedan a las funcionalidades principales.

### 4. Interfaz de Usuario

- La interfaz es el punto de interacción principal del usuario y está diseñada para ser clara y fácil de usar:

Selección de modo:

- Un desplegable creado con st.selectbox("Selecciona el modo", ["Predicción de la demanda", "Informe de la predicción"]) permite al usuario elegir entre los dos modos principales.

- El valor seleccionado se almacena en una variable, por ejemplo, mode, que determina qué funcionalidad se ejecuta.


Títulos estilizados:

- Para cada modo, se muestra un título usando st.markdown con HTML personalizado. Por ejemplo:<h1 style='color: #0071CE; text-align: center;'>Predicción de la demanda</h1>

- El color #0071CE es un azul característico de Walmart, y text-align: center centra el texto para un diseño limpio y profesional.

- Esta interfaz actúa como un "menú" que guía al usuario hacia las funcionalidades específicas de predicción o generación de informes.

### 5. Modo Predicción de la Demanda

- Este modo permite a los usuarios predecir la demanda de un producto en una tienda específica para junio de 2016. Aquí está el detalle exhaustivo:


Selección en la barra lateral:

- Producto: Un desplegable (st.sidebar.selectbox) muestra los productos disponibles, obtenidos de las claves del diccionario product_to_stores. Por ejemplo, opciones como FOODS_2_197, FOODS_3_090.

- Tienda: Otro desplegable muestra las tiendas asociadas al producto seleccionado, como CA_1, CA_2, extraídas de product_to_stores[producto]. Se transforma el ID a un formato legible, como "Tienda 1" para CA_1.

- Estado: Un mapeo predefinido (por ejemplo, {"CA": "California"}) convierte el prefijo de la tienda (CA) en el nombre del estado, mostrado con st.sidebar.write.

- Período: Los desplegables para mes ("Junio") y año (2016) están deshabilitados (disabled=True) porque la predicción está fija en junio de 2016.



Mapa interactivo:

- Generación: Usa px.choropleth con un DataFrame que contiene códigos de estados de EE. UU. (por ejemplo, CA para California) y una columna de color basada en la selección (1 para el estado seleccionado, 0 para otros).

- Estilización: Configuraciones como color_continuous_scale=["gray", "#0071CE"], scope="usa", y ajustes de márgenes (margin={"r":0,"t":0,"l":0,"b":0}) crean un mapa profesional y enfocado.

- Visualización: Se muestra con st.plotly_chart.



Predicción y visualización:

- Carga: Se carga el modelo con load_model(f"Final_model_XGBOOST_{producto}_{tienda}.sav") y los datos con load_data().

- Filtrado: Los datos se filtran para el producto y tienda seleccionados, y se extrae la demanda real de mayo de 2016.

- Preparación: Se crean características para junio de 2016 (retrasos basados en meses anteriores, media móvil, dummy de mes = 6).

- Predicción: El modelo predice log(1 + demanda); se usa np.expm1 para obtener la demanda en unidades originales.

- Gráfico: Un gráfico de barras con go.Bar compara la demanda real (mayo) y predicha (junio), con colores como #0071CE y #FFC107, y efectos 3D mediante opacity y width.

- Este modo ofrece una experiencia interactiva y visualmente atractiva para explorar predicciones.

### 6. Modo Informe de la Predicción

- Este modo genera un informe PDF detallado para múltiples selecciones:

Selección múltiple:

- Estado: Desplegable con opciones como "California".

- Tiendas: Multiseleccionador (st.multiselect) con tiendas del estado (por ejemplo, "Tienda 1 (California)").

- Productos: Multiseleccionador con productos disponibles para las tiendas seleccionadas.


Generación del informe:

- Botón: st.button("Generar Informe") inicia el proceso.

Procesamiento:

Para cada combinación válida:

- Carga modelo y datos.

- Predice demanda de junio de 2016 y compara con mayo de 2016.

- Calcula el error porcentual: abs(real - predicha) / real * 100.

- Genera y guarda un gráfico de barras con Matplotlib.


PDF:

- Usa reportlab con canvas.Canvas en un buffer BytesIO.

- Agrega encabezado, tabla con Table y gráficos con canvas.drawImage.

- Se descarga con st.download_button.
